In [30]:
import tensorflow as tf
import string,os,glob
import tensorflow.keras.callbacks
from tensorflow.keras.applications import MobileNet

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import load_img
import pickle
from time import time
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional,Concatenate,Embedding, TimeDistributed, Dense, RepeatVector,Activation, Flatten, Reshape, concatenate, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import random,copy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [31]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [32]:
base_folder = '/content/drive/My Drive/20FallMLfinal/Project'
data_folder = '/content/drive/My Drive/20FallMLfinal/Project/Data'
#where each line in the hypothesis file is a generated sentence and the corresponding lines across the reference files are ground truth reference sentences for the corresponding hypothesis.


In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-xr1st_e5
  Running command git clone -q https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-xr1st_e5
     |████████████████████████████████| 471kB 12.5MB/s 
  Created wheel for nlg-eval: filename=nlg_eval-2.3-cp36-none-any.whl size=68175137 sha256=a7efe0071bd6eddabc1dec532eae16df267460477190b4cf0e802f431e1d822b
  Stored in directory: /tmp/pip-ephem-wheel-cache-krs4_syy/wheels/a5/7c/fd/f312beca2adcc3f49cb40570730658dad37bb5709f5d237a56
  Created wheel for psutil: filename=psutil-5.7.3-cp36-cp36m-linux_x86_64.whl size=281512 sha256=c4efc964bfb7c0c22365dab5e0ffa8e703b4a56eb686ab9e0230a8751629d2a2
  Stored in directory: /root/.cache/pip/wheels/42/32/da/8b12fd6b138c733efd03cfde6c6c8191a32842f9e82aa45fbf
Successfully built nlg-eval psutil
  Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
!nlg-eval --setup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 530 chunks/s]
dictionary.txt: 550 chunks [00:00, 662 chunks/s]
uni_skip.npz: 100% 634/634 [00:23<00:00, 27.5 chunks/s]
uni_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 523 chunks/s]
bi_skip.npz: 100% 276/276 [00:10<00:00, 27.0 chunks/s]
bi_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 2.21k chunks/s]
multi-bleu.perl: 100% 1.00/1.00 [00:00<00:00, 678 chunks/s]
utable.npy: 100% 2.23k/2.23k [01:17<00:00, 28.8 chunks/s]
btable.npy: 100% 2.23k/2.23k [01:31<00:00, 24.3 chunks/s]
glove.6B.zip: 100% 823/823 [06:30<00:00, 2.11 chunks/s]
2020-11-30 01:36:28,551 : MainThread : INFO : 400000 lines with 300 dimensions
2020-11-30 01:36:31,797 : MainThread : INFO : Model /root/.cache/nlgeval/glove.6B.300d.model.t

In [ ]:
#get test captions file
captiondic={}
captionf=open("/content/drive/MyDrive/20FallMLfinal/Project/Data/GCCdata_text/captions.txt","r",encoding="UTF-8")
for l in captionf:
  tokens=l.split(",")
  captiondic[tokens[0]]=tokens[1]
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/1-sized-bat1024-ep100.csv"
yhatf=open(yhatpath,"r",encoding="UTF-8")
refstr=""
reflist=[]
for l in yhatf:
  tokens=l.split(",")
  refstr+=tokens[0]+","+captiondic[tokens[0]].strip()+"\n"
  reflist.append(tokens[0])
testref="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/ref.csv"
testreff=open(testref,"w",encoding="UTF-8")
testreff.write(refstr)
testreff.close()


In [ ]:
len(reflist)

773

In [ ]:
#How to use nlgeval:
#https://github.com/Maluuba/nlg-eval

In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/1-sized-bat1024-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.123297
Bleu_2: 0.048483
Bleu_3: 0.022688
Bleu_4: 0.013190
METEOR: 0.117850
ROUGE_L: 0.121181
CIDEr: 0.169844


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


SkipThoughtsCosineSimilarity: 0.404675
EmbeddingAverageCosineSimilarity: 0.735723
EmbeddingAverageCosineSimilairty: 0.735723
VectorExtremaCosineSimilarity: 0.348702
GreedyMatchingScore: 0.629704


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/2-sized-bat1024-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.119368
Bleu_2: 0.052630
Bleu_3: 0.028511
Bleu_4: 0.018149
METEOR: 0.121237
ROUGE_L: 0.125281
CIDEr: 0.191312
SkipThoughtsCosineSimilarity: 0.409429
EmbeddingAverageCosineSimilarity: 0.727480
EmbeddingAverageCosineSimilairty: 0.727480
VectorExtremaCosineSimilarity: 0.359284
GreedyMatchingScore: 0.639330


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/3-sized-bat1024-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.124360
Bleu_2: 0.053086
Bleu_3: 0.026226
Bleu_4: 0.015191
METEOR: 0.120743
ROUGE_L: 0.124366
CIDEr: 0.191809
SkipThoughtsCosineSimilarity: 0.406229
EmbeddingAverageCosineSimilarity: 0.733962
EmbeddingAverageCosineSimilairty: 0.733962
VectorExtremaCosineSimilarity: 0.360939
GreedyMatchingScore: 0.632879


In [33]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/4-sized-bat1024-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.123601
Bleu_2: 0.052335
Bleu_3: 0.025747
Bleu_4: 0.014715
METEOR: 0.119760
ROUGE_L: 0.127016
CIDEr: 0.186652
SkipThoughtsCosineSimilarity: 0.407717
EmbeddingAverageCosineSimilarity: 0.732348
EmbeddingAverageCosineSimilairty: 0.732348
VectorExtremaCosineSimilarity: 0.355925
GreedyMatchingScore: 0.637962


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/1-sized-bat512-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.115873
Bleu_2: 0.047041
Bleu_3: 0.022006
Bleu_4: 0.012656
METEOR: 0.116522
ROUGE_L: 0.111753
CIDEr: 0.160985
SkipThoughtsCosineSimilarity: 0.394729
EmbeddingAverageCosineSimilarity: 0.727939
EmbeddingAverageCosineSimilairty: 0.727939
VectorExtremaCosineSimilarity: 0.345492
GreedyMatchingScore: 0.622261


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/2-sized-bat512-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.112705
Bleu_2: 0.048310
Bleu_3: 0.024429
Bleu_4: 0.014460
METEOR: 0.117573
ROUGE_L: 0.117299
CIDEr: 0.156533
SkipThoughtsCosineSimilarity: 0.402424
EmbeddingAverageCosineSimilarity: 0.721693
EmbeddingAverageCosineSimilairty: 0.721693
VectorExtremaCosineSimilarity: 0.350816
GreedyMatchingScore: 0.627226


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/3-sized-bat512-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.117922
Bleu_2: 0.053978
Bleu_3: 0.030207
Bleu_4: 0.020533
METEOR: 0.120620
ROUGE_L: 0.122535
CIDEr: 0.207402
SkipThoughtsCosineSimilarity: 0.403061
EmbeddingAverageCosineSimilarity: 0.727397
EmbeddingAverageCosineSimilairty: 0.727397
VectorExtremaCosineSimilarity: 0.363023
GreedyMatchingScore: 0.635862


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/4-sized-bat512-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.108679
Bleu_2: 0.043420
Bleu_3: 0.020442
Bleu_4: 0.012378
METEOR: 0.115591
ROUGE_L: 0.111049
CIDEr: 0.150996
SkipThoughtsCosineSimilarity: 0.401983
EmbeddingAverageCosineSimilarity: 0.725792
EmbeddingAverageCosineSimilairty: 0.725792
VectorExtremaCosineSimilarity: 0.352064
GreedyMatchingScore: 0.624166


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/1-sized-bat128-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.105157
Bleu_2: 0.038993
Bleu_3: 0.017829
Bleu_4: 0.009686
METEOR: 0.111512
ROUGE_L: 0.103260
CIDEr: 0.134278
SkipThoughtsCosineSimilarity: 0.393969
EmbeddingAverageCosineSimilarity: 0.711589
EmbeddingAverageCosineSimilairty: 0.711589
VectorExtremaCosineSimilarity: 0.328259
GreedyMatchingScore: 0.613272


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/2-sized-bat128-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.110737
Bleu_2: 0.047654
Bleu_3: 0.023219
Bleu_4: 0.013393
METEOR: 0.115943
ROUGE_L: 0.116841
CIDEr: 0.155438
SkipThoughtsCosineSimilarity: 0.395568
EmbeddingAverageCosineSimilarity: 0.713968
EmbeddingAverageCosineSimilairty: 0.713968
VectorExtremaCosineSimilarity: 0.333184
GreedyMatchingScore: 0.627835


In [ ]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/3-sized-bat128-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.109329
Bleu_2: 0.042527
Bleu_3: 0.019881
Bleu_4: 0.011587
METEOR: 0.113810
ROUGE_L: 0.113574
CIDEr: 0.143923
SkipThoughtsCosineSimilarity: 0.400674
EmbeddingAverageCosineSimilarity: 0.709967
EmbeddingAverageCosineSimilairty: 0.709967
VectorExtremaCosineSimilarity: 0.332768
GreedyMatchingScore: 0.622869


In [28]:
#whole file
from nlgeval import compute_metrics
yhatpath="/content/drive/MyDrive/20FallMLfinal/Project/workspace/zzx/zzx-model/GCC/4-sized-bat128-ep100.csv"
#metrics_dict = compute_metrics(hypothesis='examples/hyp.txt',references=['examples/ref1.txt', 'examples/ref2.txt'])
metrics_dict = compute_metrics(hypothesis=yhatpath,references=[testref])

Bleu_1: 0.107812
Bleu_2: 0.044240
Bleu_3: 0.021787
Bleu_4: 0.012700
METEOR: 0.112882
ROUGE_L: 0.112224
CIDEr: 0.149707
SkipThoughtsCosineSimilarity: 0.394296
EmbeddingAverageCosineSimilarity: 0.715319
EmbeddingAverageCosineSimilairty: 0.715319
VectorExtremaCosineSimilarity: 0.328023
GreedyMatchingScore: 0.621162


In [29]:
#ind sentence
from nlgeval import compute_individual_metrics
metrics_dict = compute_individual_metrics(references, hypothesis)

NameError: ignored